## J1 : Cours de ia predilective 

In [ ]:
#!pip3 install torch torchvision

In [ ]:
!where pip
import sys
sys.executable

In [ ]:
import numpy as np

In [ ]:
import torch 

In [ ]:
#data= [[1,2], [3,4]]

In [ ]:
#x_data.size()

In [ ]:
#x_data = torch.tensor(data)
#x_data

In [ ]:
#np_array = np.array(data)


In [ ]:
w = torch.tensor([2.0], requires_grad=True)
b = torch.tensor([1.0], requires_grad=True)
x = torch.tensor([3.0])

In [ ]:
y_pred = w*x+b
print("y_pred:", y_pred.item())

In [ ]:
y_true = torch.tensor([10.0])
loss= (y_pred - y_true)**2
print("Loss:", loss.item())

In [ ]:
print("Avant backward: w.grad =", {w.grad})

In [ ]:
loss.backward()

In [ ]:
print("Après backward: w.grad =", {w.grad})
print("Après backward: b.grad =", {b.grad})

In [ ]:
#update manuel des poids
learning_rate = 0.01
with torch.no_grad(): #désactive le suivi des gradients pour les opérations de mise à jour w=w+alpha*x
    w -= learning_rate * w.grad
    b -= learning_rate * b.grad

In [ ]:
print("Nouveaux paramètres: w =", w, ", b =", b)

In [ ]:
# Réinitialiser les gradients !! Important
w.grad.zero_()
b.grad.zero_()
print("Après remise à zéro: w.grad =", w.grad, ", b.grad =", b.grad)

In [ ]:
import torch.nn as nn

percepton = nn.Linear(1, 1)

#voir les poids initiaux
print("poids:", percepton.weight)
print("biais : ", percepton.bias)

In [ ]:
#valeur de input x
#forward
x = torch.tensor([[3.0]]) # Shape (batch_size, features)
print("input:", x)
y_pred = percepton(x)
print(y_pred)

### théoreme centrale de limite (?)

In [ ]:
#Loss
y_true = torch.tensor([10.0])
criterion = nn.MSELoss()
loss = criterion(y_pred, y_true)

In [ ]:
loss.backward()

In [ ]:
print("Gradient du pois =", percepton.weight.grad)
print("Gradient du biais =", percepton.bias.grad)

###### pour la partie de en haut j'ai pas eu les meme res que le prof

In [ ]:
import torch.optim as optim
optimizer = optim.SGD(percepton.parameters(), lr=0.01)

In [ ]:
optimizer.step() #met à jour les poids automatiquement

In [ ]:
print("Nouveaux poids:", percepton.weight, "Nouveau biais:", percepton.bias)

###### pour la partie de en haut j'ai pas eu les meme res que le prof

#### On va créer des models de neuronnes

In [ ]:
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 4)
        self.fc2 = nn.Linear(4, 3)
        self.fc3 = nn.Linear(3, 1)
        self.relu = nn.ReLU()
        self.sigmoid= nn.Sigmoid()

    def forward(self, x): 
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x


In [ ]:
model = SimpleNet()

In [ ]:
model

In [ ]:
x = torch.tensor([1.0, 2.0])
y = torch.tensor([5.0])
y_pred = model(x)
loss = nn.MSELoss()(y_pred, y)

loss.backward()

In [ ]:
# !pip3 install matplotlib

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Compose, Normalize
import matplotlib.pyplot as plt

In [ ]:
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor() #compose to tensor et normalize
)

In [ ]:
training_dataloader = DataLoader(training_data, batch_size=64, shuffle=True, pin_memory=True, num_workers=4)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=False, pin_memory=True, num_workers=4)

In [ ]:
print(training_data.classes)
print(training_data.data.size())

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc = nn.Sequential(
            nn.Linear(28*28, 512), # taille image (28*28) 512
            nn.ReLU(),
            nn.Linear(512, 128), # 512 - 128
            nn.ReLU(),
            nn.Linear(128, 10), # 128
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.fc(x)
        return logits

In [ ]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
model = NeuralNetwork().to(device)
print(model)

In [ ]:
x=torch.rand(1,28,28,device=device)
logits=model(x)

In [ ]:
logits

In [ ]:
pred_probab = nn.Softmax(dim=1)(logits)
pred_probab

In [ ]:
loss_fn= nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
#pf tips
tuple_tets = (1,2,"a")
a,b,c = tuple_tets
print()

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch_idx, batch_value in enumerate(dataloader): 
        X, y = batch_value
        X, y = X.to(device), y.to(device)

        # Forward pass
        pred = model(X)
        loss = loss_fn(pred, y)

        # Background pass
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


        if batch_idx % 100 == 0:
            loss, current = loss.item(), (batch_idx+1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0 
    with torch.no_grad():
        for X, y in dataloader :
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss+= loss_fn(pred, y).item()
            correct+= (pred.argmax(1)==y).type(torch.float).sum().item()

    test_loss/= num_batches
    correct/= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 12
for t in range(epochs):
    print(f"Epochs {t+1}\n-------------------------------")
    train(training_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")


##### pythorch tutoriel : https://docs.pytorch.org/tutorials/beginner/onnx/export_simple_model_to_onnx_tutorial.html

In [ ]:
#!pip3 install --upgrade onnx onnxscript

In [ ]:
example_inputs = (torch.randn(1, 1, 28, 28),)
model.to("cpu")
onnx_program = torch.onnx.export(model, example_inputs, dynamo=True)
onnx_program.save("model.onnx")

##### html ccs js githubpage onnxruntime
créer un canva, recup la taille, transfor le canva en tensor onnx coté js etc etc

###### email du prof : nathan.vidal@ext.devinci.fr

## J2 : IA pour le web

* CNN (filtre + MLP ) -> new architecture
* Dropout -> ajouter dans lancien, cest bien dans entrainment mais pas phase de test
* Tensorboard -> faire des graphes
* Normalization(transformation et modèle) -> si on fait normalization sur python et faut faire normalization sur js
##### Cette première partie ce fera en autonomie, et à rendre ce soir maximum grand maximum avant 00h, (youtube : DeepIA)

===============================
##### bonus
* Custom Dataset
* Augmentation data (dans transform)
* Transfer Learning (en utilisant un modèle déja fait, pour voir si ça a changé le resultat)

### Modele CNN

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import Compose, ToTensor, Normalize
from torch.utils.tensorboard import SummaryWriter


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cpu')

In [4]:
transform = Compose([
    ToTensor(),
    Normalize((0.1307,), (0.3081,))
])


In [5]:
train_data = datasets.MNIST(root="data", train=True, download=True, transform=transform)
test_data  = datasets.MNIST(root="data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_data, batch_size=64, shuffle=False)

In [6]:
conv1 = nn.Conv2d(1, 6, 3, stride=1, padding=1)
conv2 = nn.Conv2d(6, 16, 3, stride=1, padding=1)


In [7]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 6, kernel_size=3, stride=1, padding=1)   # 28x28 → 28x28
        self.conv2 = nn.Conv2d(6, 16, kernel_size=3, stride=1, padding=1)  # 14x14 → 14x14

        self.pool = nn.MaxPool2d(2, 2)  

        self.dropout = nn.Dropout(0.5)   
        self.relu = nn.ReLU()

        
        self.fc1 = nn.Linear(16 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.relu(self.conv1(x))   
        x = self.pool(x)               
        x = self.relu(self.conv2(x))   
        x = self.pool(x)              

        x = torch.flatten(x, 1)        

        x = self.dropout(x)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)

        return x


In [8]:
model = SimpleCNN().to(device)
model


SimpleCNN(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.5, inplace=False)
  (relu): ReLU()
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)


In [10]:
writer = SummaryWriter(log_dir="runs/mnist_cnn")

In [11]:
def train(model, loader, criterion, optimizer, epoch):
    model.train()
    running_loss = 0

    for i, (X, y) in enumerate(loader):
        X, y = X.to(device), y.to(device)

        optimizer.zero_grad()
        output = model(X)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % 200 == 0:
            print(f"Epoch {epoch} | Batch {i} | Loss {loss.item():.4f}")

    avg_loss = running_loss / len(loader)
    writer.add_scalar("Loss/train", avg_loss, epoch)


In [12]:
def test(model, loader, criterion, epoch):
    model.eval()
    correct = 0
    total_loss = 0

    with torch.no_grad():
        for X, y in loader:
            X, y = X.to(device), y.to(device)
            output = model(X)
            loss = criterion(output, y)
            total_loss += loss.item()
            pred = output.argmax(1)
            correct += (pred == y).sum().item()

    accuracy = correct / len(loader.dataset)
    avg_loss = total_loss / len(loader)

    writer.add_scalar("Loss/test", avg_loss, epoch)
    writer.add_scalar("Accuracy/test", accuracy, epoch)

    print(f"Test Accuracy: {accuracy*100:.2f}%")
    return accuracy


In [13]:
EPOCHS = 5

for epoch in range(1, EPOCHS+1):
    train(model, train_loader, criterion, optimizer, epoch)
    test(model, test_loader, criterion, epoch)

writer.close()


Epoch 1 | Batch 0 | Loss 2.2997
Epoch 1 | Batch 200 | Loss 0.2456
Epoch 1 | Batch 400 | Loss 0.3198
Epoch 1 | Batch 600 | Loss 0.1381
Epoch 1 | Batch 800 | Loss 0.0643
Test Accuracy: 97.70%
Epoch 2 | Batch 0 | Loss 0.2030
Epoch 2 | Batch 200 | Loss 0.1089
Epoch 2 | Batch 400 | Loss 0.0240
Epoch 2 | Batch 600 | Loss 0.1281
Epoch 2 | Batch 800 | Loss 0.0460
Test Accuracy: 98.39%
Epoch 3 | Batch 0 | Loss 0.1168
Epoch 3 | Batch 200 | Loss 0.1366
Epoch 3 | Batch 400 | Loss 0.0746
Epoch 3 | Batch 600 | Loss 0.0796
Epoch 3 | Batch 800 | Loss 0.0499
Test Accuracy: 98.32%
Epoch 4 | Batch 0 | Loss 0.0522
Epoch 4 | Batch 200 | Loss 0.0445
Epoch 4 | Batch 400 | Loss 0.0327
Epoch 4 | Batch 600 | Loss 0.0160
Epoch 4 | Batch 800 | Loss 0.1167
Test Accuracy: 98.80%
Epoch 5 | Batch 0 | Loss 0.0674
Epoch 5 | Batch 200 | Loss 0.0242
Epoch 5 | Batch 400 | Loss 0.0591
Epoch 5 | Batch 600 | Loss 0.0408
Epoch 5 | Batch 800 | Loss 0.0376
Test Accuracy: 98.89%


In [14]:
model.eval()
dummy = torch.randn(1, 1, 28, 28).to(device)

torch.onnx.export(
    model,
    dummy,
    "simple_cnn.onnx",
    input_names=["input"],
    output_names=["output"],
    opset_version=17
)

print("ONNX exporté")

W1211 16:30:55.913000 2920 Lib\site-packages\torch\onnx\_internal\exporter\_compat.py:114] Setting ONNX exporter to use operator set version 18 because the requested opset_version 17 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features


[torch.onnx] Obtain model graph for `SimpleCNN([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `SimpleCNN([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...


The model version conversion is not supported by the onnxscript version converter and fallback is enabled. The model will be converted using the onnx C API (target version: 17).


[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
ONNX exporté
